In [1]:
import pandas as pd

In [13]:
runkeeper_file = 'datasets/cardioActivities.csv'

In [10]:
pd.read_csv(runkeeper_file, parse_dates=['Date'], index_col='Date')

In [14]:
display(df_activities.sample(3))


,Activity Id,Type,Route Name,Distance (km),Duration,Average Pace,Average Speed (km/h),Calories Burned,Climb (m),Average Heart Rate (bpm),Friend's Tagged,Notes,GPX File
Date,,,,,,,,,,,,,
2013-12-04 18:36:22,0a1d041d-9ea5-438a-a2df-dcac74a33116,Running,NaN,12.48,1:10:42,5:40,10.59,896.000000,69,NaN,NaN,NaN,2013-12-04-183622.gpx
2013-02-22 19:23:38,df659987-77a5-4c7b-8633-74dbb15d3390,Running,NaN,6.32,35:58,5:41,10.55,454.000000,35,NaN,NaN,NaN,2013-02-22-192338.gpx
2016-07-10 16:35:25,85014381-7900-49e2-88f0-42c75e0954c1,Running,NaN,19.27,1:49:48,5:42,10.53,1323.999999,319,142.0,NaN,TomTom MySports Watch,2016-07-10-163525.gpx


In [22]:
df_activities.tail()

,Type,Distance (km),Duration,Average Pace,Average Speed (km/h),Climb (m),Average Heart Rate (bpm)
Date,,,,,,,
2012-08-28 07:06:57,Walking,1.57,13:39,8:41,6.91,7,NaN
2012-08-24 12:59:42,Walking,1.48,17:56,12:09,4.94,12,NaN
2012-08-24 10:12:16,Walking,1.49,13:43,9:14,6.49,9,NaN
2012-08-24 08:13:12,Running,3.15,16:00,5:05,11.82,17,NaN
2012-08-22 18:53:54,Running,5.69,31:08,5:29,10.95,32,NaN


In [8]:
df_activities.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 508 entries, 2018-11-11 14:05:12 to 2012-08-22 18:53:54
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Activity Id               508 non-null    object 
 1   Type                      508 non-null    object 
 2   Route Name                1 non-null      object 
 3   Distance (km)             508 non-null    float64
 4   Duration                  508 non-null    object 
 5   Average Pace              508 non-null    object 
 6   Average Speed (km/h)      508 non-null    float64
 7   Calories Burned           508 non-null    float64
 8   Climb (m)                 508 non-null    int64  
 9   Average Heart Rate (bpm)  294 non-null    float64
 10  Friend's Tagged           0 non-null      float64
 11  Notes                     231 non-null    object 
 12  GPX File                  504 non-null    object 
dtypes: float64(5), int64(1), obj

### Data Preprocessing

In [16]:
# Defining the list of columns to be deleted
cols_to_drop = ['Friend\'s Tagged','Route Name','GPX File','Activity Id','Calories Burned', 'Notes']


In [17]:
# Dropping the unnecessary columns

df_activities.drop(cols_to_drop, axis=1, inplace=True)

In [18]:
# Counting the types of training activities

display(df_activities.Type.value_counts())

Running    459
Cycling     29
Walking     18
Other        2
Name: Type, dtype: int64

In [19]:
# Renaming the feature 'Other' type to 'Unicycling'

df_activities['Type'] = df_activities['Type'].str.replace('Other', 'Unicycling', regex=False)

In [20]:
# Count missing values for each column

df_activities.isnull().sum(axis=0)

Type                          0
Distance (km)                 0
Duration                      0
Average Pace                  0
Average Speed (km/h)          0
Climb (m)                     0
Average Heart Rate (bpm)    214
dtype: int64

### Handling Missing Values

We can clearly see from the last output,that there are 214 missing entries for average heart rate in bpm.
We can fill in the missing values with an average value. This process is called mean imputation. When we do imputation to fill in missing values,we must be aware of the fact that the average heart rate varies for different activities.So we need to filter the dataframes by Type and calculate each activity's average heart rate, and then replace in the missing values with those means.

In [25]:
# calculating means for heart rate of different types of activities mentioned

mean_hour_run = df_activities[df_activities['Type']=='Running']['Average Heart Rate (bpm)'].mean()

mean_hour_walk = df_activities[df_activities['Type']=='Walking']['Average Heart Rate (bpm)'].mean()

mean_hour_cycle = df_activities[df_activities['Type']=='Cycling']['Average Heart Rate (bpm)'].mean()

In [26]:
#copying the splitted dataframes for simplified task
df_run = df_activities[df_activities['Type'] == 'Running'].copy()

df_walk = df_activities[df_activities['Type'] == 'Walking'].copy()

df_cycle = df_activities[df_activities['Type'] == 'Cycling'].copy()

In [29]:
# Filling missing values with counted mean values
df_walk['Average Heart Rate (bpm)'].fillna(float(mean_hour_walk), inplace=True)

df_run['Average Heart Rate (bpm)'].fillna(int(mean_hour_run), inplace=True)

df_cycle['Average Heart Rate (bpm)'].fillna(int(mean_hour_cycle), inplace=True)

In [30]:
#count the missing values
df_run.isnull().sum()

Type                        0
Distance (km)               0
Duration                    0
Average Pace                0
Average Speed (km/h)        0
Climb (m)                   0
Average Heart Rate (bpm)    0
dtype: int64